## Pandas怎样新增数据列？
#### 在进行数据分析时，经常需要按照一定条件创建新的数据列，然后进行进一步分析。

1. 直接赋值
2. df.apply方法
3. df.assign方法
4. 按条件选择分组分别赋值

In [2]:
import pandas as pd

## 0、读取csv数据到DataFrame

In [12]:
fpath="./datas/beijing_tianqi/beijing_tianqi_2018.csv"
df=pd.read_csv(fpath,encoding='gbk')

In [13]:
df.head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel
0,2018-01-01,3℃,-6℃,晴~多云,东北风,1-2级,59,良,2
1,2018-01-02,2℃,-5℃,阴~多云,东北风,1-2级,49,优,1
2,2018-01-03,2℃,-5℃,多云,北风,1-2级,28,优,1
3,2018-01-04,0℃,-8℃,阴,东北风,1-2级,28,优,1
4,2018-01-05,3℃,-6℃,多云~晴,西北风,1-2级,50,优,1


## 1、直接赋值的方法

#### 实列：清理温度列，变成数字类型

In [14]:
# 替换掉温度的后缀℃
df.loc[:,"bWendu"]=df["bWendu"].str.replace("℃","").astype('int32')
df.loc[:,"yWendu"]=df["yWendu"].str.replace("℃","").astype('int32')

In [15]:
df.head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel
0,2018-01-01,3,-6,晴~多云,东北风,1-2级,59,良,2
1,2018-01-02,2,-5,阴~多云,东北风,1-2级,49,优,1
2,2018-01-03,2,-5,多云,北风,1-2级,28,优,1
3,2018-01-04,0,-8,阴,东北风,1-2级,28,优,1
4,2018-01-05,3,-6,多云~晴,西北风,1-2级,50,优,1


#### 实列：计算温差

In [16]:
# 注意：df["bWendu"]其实是一个Series,后面的减法返回的是Series
df.loc[:,"wencha"]=df["bWendu"]-df["yWendu"]

In [17]:
df.head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel,wencha
0,2018-01-01,3,-6,晴~多云,东北风,1-2级,59,良,2,9
1,2018-01-02,2,-5,阴~多云,东北风,1-2级,49,优,1,7
2,2018-01-03,2,-5,多云,北风,1-2级,28,优,1,7
3,2018-01-04,0,-8,阴,东北风,1-2级,28,优,1,8
4,2018-01-05,3,-6,多云~晴,西北风,1-2级,50,优,1,9


## 2、df.apply方法

### (1).基础作用：对DataFrame按指定轴（axis)，逐行/逐列调用函数，把每一行/列数据（Series类型）传给函数处理，再整合结果。
### (2).关键参数 & 运行逻辑：
#### · axis参数：默认的axis=0，按列处理；如果axis=1，按行处理。
#### · 函数传递：自定义函数（如：get_wendu_type）会接收Series类型的参数（每行/列数据），函数里通过索引（列名或行索引）取数据判断，返回结果。

#### 实列：添加一列温度类型：
####   1.如果最高温度大于33度就是高温
####   2.低于-10度就是低温
####   3.否则是常温

In [18]:
def get_wendu_type(x):
    if x["bWendu"]>33:
        return '高温'
    if x["yWendu"]<-10:
        return '低温'
    return '常温'

# 注意：需要设置axis==1，这是series的index是columns
df.loc[:,"wendu_type"]=df.apply(get_wendu_type,axis=1)

In [19]:
# 查看温度类型的计数
df["wendu_type"].value_counts()

wendu_type
常温    11
Name: count, dtype: int64

## 3、df.assign方法

### (1).基础作用：给DataFrame新增一列或多列，不修改原DataFrame,而是返回包含新列的新DataFrame（原数据保留）。
### (2).关键特点：
#### · 链式调用良好：因为返回新对象，可接着对结果继续调用其他Pandas方法，让代码更简洁。
#### · 同时新增多列。

#### 实例：将温度从摄氏度变成华氏度

In [20]:
# 可以同时添加多个新的列
df.assign(
    yWendu_huashi = lambda x : x["yWendu"]*9/5+32,
    # 摄氏度转华氏度
    bWendu_huashi = lambda x : x["bWendu"]*9/5+32
)

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel,wencha,wendu_type,yWendu_huashi,bWendu_huashi
0,2018-01-01,3,-6,晴~多云,东北风,1-2级,59,良,2,9,常温,21.2,37.4
1,2018-01-02,2,-5,阴~多云,东北风,1-2级,49,优,1,7,常温,23.0,35.6
2,2018-01-03,2,-5,多云,北风,1-2级,28,优,1,7,常温,23.0,35.6
3,2018-01-04,0,-8,阴,东北风,1-2级,28,优,1,8,常温,17.6,32.0
4,2018-01-05,3,-6,多云~晴,西北风,1-2级,50,优,1,9,常温,21.2,37.4
5,2018-01-06,2,-5,多云~阴,西南风,1-2级,32,优,1,7,常温,23.0,35.6
6,2018-01-07,2,-4,阴~多云,西南风,1-2级,59,良,2,6,常温,24.8,35.6
7,2018-01-08,2,-6,晴,西北风,4-5级,50,优,1,8,常温,21.2,35.6
8,2018-01-09,1,-8,晴,西北风,3-4级,34,优,1,9,常温,17.6,33.8
9,2018-01/10,-2,-10,晴,西北风,1-2级,26,优,1,8,常温,14.0,28.4


## 4、按条件选择分组分别赋值
#### 按条件先选择数据，然后对这部分数据赋值新列

#### 实列：高低温差大于10度，则认为温差大

In [29]:
# 先创建空列（这是第一种创建新列的方法）
df["wencha_type"]=''

df.loc[df["bWendu"]-df["yWendu"]>10,"wencha_type"]="温差大"
df.loc[df["bWendu"]-df["yWendu"]<=10,"wencha_type"]="温差正常"

In [30]:
df["wencha_type"].value_counts()

wencha_type
温差正常    11
Name: count, dtype: int64

In [31]:
df.head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel,wencha,wendu_type,wencha_type,wenncha_type
0,2018-01-01,3,-6,晴~多云,东北风,1-2级,59,良,2,9,温差正常,温差正常,NaN
1,2018-01-02,2,-5,阴~多云,东北风,1-2级,49,优,1,7,温差正常,温差正常,温差正常
2,2018-01-03,2,-5,多云,北风,1-2级,28,优,1,7,温差正常,温差正常,温差正常
3,2018-01-04,0,-8,阴,东北风,1-2级,28,优,1,8,温差正常,温差正常,NaN
4,2018-01-05,3,-6,多云~晴,西北风,1-2级,50,优,1,9,温差正常,温差正常,NaN
